In [3]:
import numpy as np

class HiddenMarkovModel:
    def __init__(self, states, start_prob, trans_prob, emis_prob):
        self.states = states
        self.start_prob = start_prob
        self.trans_prob = trans_prob
        self.emis_prob = emis_prob

    def forward(self, obs):
        fwd = [{}]
        # Initialize forward probabilities
        for s in self.states:
            fwd[0][s] = self.start_prob[s] * self.emis_prob[s][obs[0]]

        # Forward pass
        for t in range(1, len(obs)):
            fwd.append({})
            for s in self.states:
                fwd[t][s] = sum(fwd[t-1][s0] * self.trans_prob[s0][s] * self.emis_prob[s][obs[t]] for s0 in self.states)

        return fwd

    def backward(self, obs):
        bwd = [{} for _ in range(len(obs))]
        # Initialize backward probabilities
        for s in self.states:
            bwd[-1][s] = 1

        # Backward pass
        for t in range(len(obs) - 2, -1, -1):
            for s in self.states:
                bwd[t][s] = sum(bwd[t + 1][s1] * self.trans_prob[s][s1] * self.emis_prob[s1][obs[t + 1]] for s1 in self.states)

        return bwd

    def baum_welch_train(self, observations, iterations=100):
        for _ in range(iterations):
            fwd = self.forward(observations)
            bwd = self.backward(observations)

            # Estimate gamma and xi
            gamma = [
                {s: fwd[t][s] * bwd[t][s] / sum(fwd[t][s0] * bwd[t][s0] for s0 in self.states) for s in self.states}
                for t in range(len(observations))
            ]

            xi = []
            for t in range(len(observations) - 1):
                xi_t = {}
                denom = sum(fwd[t][s0] * self.trans_prob[s0][s1] * self.emis_prob[s1][observations[t + 1]] * bwd[t + 1][s1] for s0 in self.states for s1 in self.states)
                for s0 in self.states:
                    xi_t[s0] = {}
                    for s1 in self.states:
                        xi_t[s0][s1] = (fwd[t][s0] * self.trans_prob[s0][s1] * self.emis_prob[s1][observations[t + 1]] * bwd[t + 1][s1]) / denom
                xi.append(xi_t)

            # Re-estimate the transition and emission probabilities
            for s in self.states:
                self.start_prob[s] = gamma[0][s]

                for s1 in self.states:
                    self.trans_prob[s][s1] = sum(xi[t][s][s1] for t in range(len(observations) - 1)) / sum(gamma[t][s] for t in range(len(observations) - 1))

                for o in range(1, 4):  # Emission probabilities for 1, 2, 3 ice creams
                    self.emis_prob[s][o] = sum(gamma[t][s] for t in range(len(observations)) if observations[t] == o) / sum(gamma[t][s] for t in range(len(observations)))


states = ['Cold', 'Hot']
start_prob = {'Cold': 0.5, 'Hot': 0.5}
trans_prob = {'Cold': {'Cold': 0.5, 'Hot': 0.5}, 'Hot': {'Cold': 0.4, 'Hot': 0.6}}
emis_prob = {'Cold': {1: 0.5, 2: 0.4, 3: 0.1}, 'Hot': {1: 0.2, 2: 0.4, 3: 0.4}}
observations = [ 3, 2, 1]

# Create HMM instance
hmm = HiddenMarkovModel(states, start_prob, trans_prob, emis_prob)

hmm.trans_prob


{'Cold': {'Cold': 0.5, 'Hot': 0.5}, 'Hot': {'Cold': 0.4, 'Hot': 0.6}}

In [ ]:
import numpy as np

class HiddenMarkovModel:
    def __init__(self, states, start_prob, trans_prob, emis_prob):
        self.states = states
        self.start_prob = start_prob
        self.trans_prob = trans_prob
        self.emis_prob = emis_prob

    def forward(self, obs):
        fwd = [{}]
        # Initialize forward probabilitie
        
        for s in self.states:
            fwd[0][s] = self.start_prob[s] * self.emis_prob[s][obs[0]]

        # Forward pass
        for t in range(1, len(obs)):
            fwd.append({})
            for s in self.states:
                fwd[t][s] = sum(fwd[t-1][s0] * self.trans_prob[s0][s] * self.emis_prob[s][obs[t]] for s0 in self.states)
        return fwd

    def viterbi(self, observations):
        T = len(observations)
        S = len(self.states)
        V = np.zeros((S, T))
        backpointer = np.zeros((S, T), dtype=int)

        # Initialization
        for s in range(S):
            V[s, 0] = self.start_prob[self.states[s]] * self.emis_prob[self.states[s]][observations[0]]
            backpointer[s, 0] = 0

        # Recursion
        for t in range(1, T):
            for s in range(S):
                transition_probs = [V[s_prev, t-1] * self.trans_prob[self.states[s_prev]][self.states[s]] for s_prev in range(S)]
                V[s, t] = max(transition_probs) * self.emis_prob[self.states[s]][observations[t]]
                backpointer[s, t] = np.argmax(transition_probs)

        # Termination
        best_last_state = np.argmax(V[:, T-1])
        best_path_prob = V[best_last_state, T-1]

        # Path reconstruction
        best_path = np.zeros(T, dtype=int)
        best_path[T-1] = best_last_state

        for t in range(T-2, -1, -1):
            best_path[t] = backpointer[best_path[t+1], t+1]

        best_state_sequence = [self.states[state_index] for state_index in best_path]

        return best_state_sequence, best_path_prob

# Define the HMM parameters
states = ['Cold', 'Hot']
start_prob = {'Cold': 0.5, 'Hot': 0.5}
trans_prob = {'Cold': {'Cold': 0.5, 'Hot': 0.5}, 'Hot': {'Cold': 0.4, 'Hot': 0.6}}
emis_prob = {'Cold': {1: 0.5, 2: 0.4, 3: 0.1}, 'Hot': {1: 0.2, 2: 0.4, 3: 0.4}}
observations = [3, 2, 1]  # Number of ice creams eaten on different days

# Create HMM instance
hmm = HiddenMarkovModel(states, start_prob, trans_prob, emis_prob)

# Run the forward pass
fwd_probabilities = hmm.forward(observations)
print("Forward probabilities:")
for t in range(len(observations)):
    print(f"Time {t+1}: {fwd_probabilities[t]}")

# Decode the observations using the Viterbi algorithm
decoded_states, path_probability = hmm.viterbi(observations)
print("Most likely sequence of states:", decoded_states)
print("Probability of this path:", path_probability)


Forward probabilities:
Time 1: {'Cold': 0.05, 'Hot': 0.2}
Time 2: {'Cold': 0.04200000000000001, 'Hot': 0.058}
Time 3: {'Cold': 0.0221, 'Hot': 0.011160000000000002}
Most likely sequence of states: ['Hot', 'Hot', 'Cold']
Probability of this path: 0.009600000000000001
